In [1]:
import json
from pprint import pprint
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import one_hot
import pandas as pd
import numpy as np
from IPython.display import display, HTML, Image
from bokeh.plotting import figure, output_file, show
from bokeh.io import output_notebook
from IPython.display import SVG
from keras.models import Model, model_from_json
from keras.models import Sequential
from keras.layers import Input, Dense, LSTM, concatenate, Reshape, Embedding
from keras.utils.vis_utils import plot_model, model_to_dot
from keras.utils import plot_model
from keras.optimizers import Adam, SGD

/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.
/anaconda3/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
source_fl_nm = '/Users/jaydeep/jaydeep_workstation/Workplace/Java/ontosim/source.json'
with open(source_fl_nm) as f:
    source_data = json.load(f)
    
target_fl_nm = '/Users/jaydeep/jaydeep_workstation/Workplace/Java/ontosim/target.json'
with open(target_fl_nm) as f:
    target_data = json.load(f)

onto_dict = [] 

for key in source_data.keys():
    words = source_data[key]['lbl'].split("_")
    for word in words:
        onto_dict.append(word.lower()) #Converting every word to lower

        
for key in target_data.keys():
    words = target_data[key]['lbl'].split("_")
    for word in words:
        onto_dict.append(word.lower()) #Converting every word to lower

#Create unique dictionary
onto_dict = list(set(onto_dict))
onto_dict[0:3]

['channel', 'lining', 'vastus lateralis']

In [3]:
onto_embedding_vocab_size = len(onto_dict)
encoded_onto_entity = [one_hot(d, onto_embedding_vocab_size)[0] for d in onto_dict]
encoded_onto_entity[0:3]

[299, 1495, 198]

In [4]:
# No need to PAD because we are trying to find embedding of words, not sentences.
# maxlen = 5
# padded_docs_oe = pad_sequences(encoded_docs_oe, maxlen=maxlen, padding='post')
# print(padded_docs_oe)

In [5]:
def embed_net():
    
    model = Sequential()
    model.add(Embedding(input_dim=onto_embedding_vocab_size,
                        output_dim=32, 
                        input_length=1))
    
    return model

In [6]:
model = embed_net()
model.compile('rmsprop', 'mse')
output_array = model.predict(encoded_onto_entity)
output_array[0][0]

array([ 0.04561409,  0.01607234, -0.0280853 , -0.04301577,  0.03786315,
       -0.02747009,  0.00582824,  0.01840984,  0.02709312, -0.01771881,
       -0.01351823,  0.02070579,  0.02625401,  0.03583862,  0.02028886,
        0.02875984, -0.02643342,  0.04025019, -0.04194695, -0.02462727,
        0.03833659, -0.04707463,  0.01439152,  0.03361202,  0.04496438,
       -0.04145938,  0.01658559, -0.03317313,  0.03591749, -0.04484371,
        0.03970841, -0.02806826], dtype=float32)

In [7]:
onto_dict_vec = {}
for idx, val in enumerate(onto_dict):
    onto_dict_vec[val] = output_array[idx][0] 

In [25]:
for key in source_data.keys():
    words = source_data[key]['lbl'].split("_")
    embed_vec = [0.0] * 32
    for word in words:
        embed_vec = embed_vec + onto_dict_vec[word.lower()]
    source_data[key]['vector'] = embed_vec.tolist()
    
for key in target_data.keys():
    words = target_data[key]['lbl'].split("_")
    embed_vec = [0.0] * 32
    for word in words:
        embed_vec = embed_vec + onto_dict_vec[word.lower()]
    target_data[key]['vector'] = embed_vec.tolist()

In [26]:
with open('source_data.json', 'w') as outfile:
    json.dump(source_data, outfile, indent=4)
    
with open('target_data.json', 'w') as outfile:
    json.dump(target_data, outfile, indent=4)